In [1]:
from datetime import datetime

In [2]:
print("Current Date/Time: ", datetime.now())

Current Date/Time:  2020-03-17 16:30:32.192877


In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

In [15]:
SEED = 1

# CUDA?
# cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("CUDA Available?", device)

# For reproducibility
torch.manual_seed(SEED)

if device:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=128, num_workers=2, pin_memory=True) if device else dict(shuffle=True, batch_size=64)

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset,** dataloader_args)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, **dataloader_args)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

CUDA Available? cuda:0
Files already downloaded and verified
Files already downloaded and verified


In [23]:
import torch.nn as nn
import torch.nn.functional as F


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.con1=nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=0), # input: 3x3 output: 32	Rec.field: 1	JumpOut: 1
            nn.ReLU(),
            nn.BatchNorm2d(16), 
         )
        
        self.con2=nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1), # input: 32  output: 32	Rec.field: 3	JumpOut: 1
            nn.ReLU(),
            nn.BatchNorm2d(32), 
         )
        
        self.con3=nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0), # input: 32  output: 30	Rec.field: 5	JumpOut: 1
            nn.ReLU(),
            nn.BatchNorm2d(64), 
         )
         
        self.pool1=nn.MaxPool2d(2,2) # input: 30  output: 15	Rec.field: 6	JumpOut: 1
         
        self.con4=nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1), # input: 15  output: 15	Rec.field: 10	JumpOut: 2
            nn.ReLU(),
            nn.BatchNorm2d(128), 
         )
         
        self.con5=nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1), # input: 15  output: 15	Rec.field: 14	JumpOut: 2
            nn.ReLU(),
            nn.BatchNorm2d(128), 
         )
         
        self.con6=nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1), # input: 15  output: 15	Rec.field: 18	JumpOut: 2
            nn.ReLU(),
            nn.BatchNorm2d(64), 
         )
         
        self.pool2=nn.MaxPool2d(2,2) # input: 7   output: 7	Rec.field: 20	JumpOut: 2
        
        self.con7=nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1), # input: 7   output: 7	Rec.field: 28	JumpOut: 4
            nn.ReLU(),
            nn.BatchNorm2d(32), 
         )
         
        self.con8=nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=1, padding=1), # input: 7   output: 7	Rec.field: 36	JumpOut: 4
            nn.ReLU(),
            nn.BatchNorm2d(16), 
         )
         
        self.con9=nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=10, kernel_size=3, stride=1, padding=1), # input: 7   output: 7	Rec.field: 44	JumpOut: 4
         )

        self.avg_pool=nn.AvgPool2d(kernel_size=7)
        
        

    def forward(self, x):
        x = self.con1(x)
        x = self.con2(x)
        x = self.con3(x)
        x = self.pool1(x)
        x = self.con4(x)
        x = self.con5(x)
        x = self.con6(x)
        x = self.pool2(x)
        x = self.con7(x)
        x = self.con8(x)
        x = self.con9(x)
        x = self.avg_pool(x)
        x = x.view(-1, 10)
        return x

In [24]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)
net = Net().to(device)
summary(net, input_size=(3, 32, 32))

cuda
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 30, 30]             448
              ReLU-2           [-1, 16, 30, 30]               0
       BatchNorm2d-3           [-1, 16, 30, 30]              32
            Conv2d-4           [-1, 32, 30, 30]           4,640
              ReLU-5           [-1, 32, 30, 30]               0
       BatchNorm2d-6           [-1, 32, 30, 30]              64
            Conv2d-7           [-1, 64, 28, 28]          18,496
              ReLU-8           [-1, 64, 28, 28]               0
       BatchNorm2d-9           [-1, 64, 28, 28]             128
        MaxPool2d-10           [-1, 64, 14, 14]               0
           Conv2d-11          [-1, 128, 14, 14]          73,856
             ReLU-12          [-1, 128, 14, 14]               0
      BatchNorm2d-13          [-1, 128, 14, 14]             256
           Conv2d-14          [-1,

In [25]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-5)

In [26]:
import torch
import torchvision
# import torchvision.transforms as transforms
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []


def train(net, device, optimizer,criterion, epoch,trainloader):
  #trainloader = dataloader_cifar10(root='./data',, split='train', batch_size=args.batch_size)
  net.train()
  pbar = tqdm(trainloader)
  correct = 0
  processed = 0
  for batch_idx, (input, labels) in enumerate(pbar):
    # get samples
    input, labels = input.to(device), labels.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    outputs = net(input)

    # Calculate loss
#     loss = F.nll_loss(outputs, target)
    loss = criterion(outputs, labels)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    _, pred = torch.max(outputs, 1)  # get the index of the max log-probability
    correct += pred.eq(labels.view_as(pred)).sum().item()
    processed += len(input)

    pbar.set_description(desc= f'Epoch={epoch} Loss={loss.item()} Batch_id={batch_idx} Training Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)
    
import torch
import torchvision
import torchvision.transforms as transforms
     
def test(net, device,criterion, testloader):
    #testloader = dataloader_cifar10(root='./data',, split='test', batch_size=dataloader_args.batch_size)
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    accuracy = 0
    classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
            #     % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        test_loss /= len(testloader.dataset)
        
        accuracy=100.00 * correct / len(testloader.dataset)
        
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                test_loss, correct, len(testloader.dataset),
                accuracy))
                
        if accuracy >=87:
            class_correct = list(0. for i in range(10))
            class_total = list(0. for i in range(10))
            with torch.no_grad():
                for data in testloader:
                    images, labels = data
                    images, labels = images.to(device), labels.to(device)
                    outputs = net(images)
                    _, predicted = torch.max(outputs, 1)
                    c = (predicted == labels).squeeze()
                    for i in range(4):
                        label = labels[i]
                        class_correct[label] += c[i].item()
                        class_total[label] += 1


            for i in range(10):
                print('Accuracy of %5s : %2d %%' % (
                    classes[i], 100 * class_correct[i] / class_total[i]))

In [27]:
for epoch in range(20):
    train(net, device, optimizer,criterion, epoch,trainloader)
    test(net, device,criterion, testloader)




  0%|                                                                    | 0/391 [00:00<?, ?it/s]

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 4.00 GiB total capacity; 530.87 MiB already allocated; 0 bytes free; 57.13 MiB cached)